# Checkpointing in FEniCSx

## Jørgen Schartum Dokken

### Simula Research Laboratory

FEniCS 23'

<div>
<p style="text-align:center;"><img src="fenics_banner.png" width="500">
</div>


In [5]:
5

5

In [2]:
print("ss")

NameError: name 'ss' is not defined